<a href="https://colab.research.google.com/github/andrewsoul16/AndrewSoul16/blob/master/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy, cv2
import os
import numpy as np
import keras
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.utils import to_categorical

# Размер входного изображения
size = 96

# Папка с обучающей выборкой
train_dir = "training"

# Цвет изображения
CLR = cv2.IMREAD_COLOR #COLOR GRAYSCALE

# Количество классов
classes = 4

def create_train_data():
  # Массивы X и Y
  train_data = []
  train_labels = []
  
  #Индекс класса
  pp = -1
  
  # По всем классам(папкам), и по всем файлам в них
  for p1 in sorted(os.listdir(train_dir)):
    if pp == -1: 
      pp=pp+1 
      continue 
    for img in os.listdir(os.path.join(train_dir, p1)):
          # Формируем путь к файлу
          path = os.path.join(os.path.join(train_dir, p1), img)
          # Изменяем размер изображения и цветовое пространство
          img = cv2.resize(cv2.imread(path, CLR), (size, size))
          # Добавляем картинку в обучающую выборку
          train_data.append(list(np.array(img)))
          # А также её индекс
          train_labels.append([pp])
    print(pp,p1)
    pp=pp+1
  
  # Формируем и выгружаем массивы
  train_labels = np.array(train_labels)
  train_data = np.array(train_data)
  return (train_data,train_labels)

#print(create_train_data())

(X_train, Y_train) = create_train_data()
Y_train_en = to_categorical(Y_train, num_classes=classes, dtype='float32')

# Конфигурируем модель сети как последовательную
def build_model():
    model = Sequential()
    
    # размером SIZE*SIZE*3. В первом блоке свёртка окном 3*3 
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(size, size, 3)))
    model.add(Activation("relu"))  # Активация ReLU
    # Подвыборка квадратом 3*3 максимума из квадрата
    model.add(MaxPooling2D(pool_size=(3, 3)))
    
    # [32, size/3, size/3]
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # [32, size/6, size/6]
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # [64, size/12, size/12]
    model.add(Conv2D(128, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # [128, size/24, size/24]
    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # [64, size/48, size/48]
    model.add(Conv2D(32, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # [32, size/96, size/96]
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1024))
    model.add(Activation("relu"))
 
    model.add(Dense(classes))
    model.add(Activation("softmax"))
    return model

model = build_model()
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.fit(X_train,Y_train_en,epochs=30, batch_size=32)

# Скачиваем скрипт для конвертации сети
!git clone https://github.com/amir-abdi/keras_to_tensorflow.git
  
# Сохраняем сеть в формате Keras
model.save("Keras_model.h5")

# Сохраняем описание узлов в описании JSON
model_json = model.to_json()
with open("JSON_model.json", "w") as json_file:
    json_file.write(model_json)

# Сохраняем описание узлов в "замороженном" виде в формате protobuf
!python keras_to_tensorflow/keras_to_tensorflow.py --input_model="Keras_model.h5" --input_model_json="JSON_model.json" --output_model="Final_model.pb"


0 Arduino UNO
1 FireFly
2 Nano Pi
3 Raspberry PI
Epoch 1/30
17/17 [==============================] - 4s 241ms/step - loss: 2.3448 - categorical_accuracy: 0.2775
Epoch 2/30
17/17 [==============================] - 4s 235ms/step - loss: 1.3444 - categorical_accuracy: 0.3410
Epoch 3/30
17/17 [==============================] - 4s 237ms/step - loss: 1.2578 - categorical_accuracy: 0.4297
Epoch 4/30
17/17 [==============================] - 4s 240ms/step - loss: 1.1827 - categorical_accuracy: 0.4605
Epoch 5/30
17/17 [==============================] - 4s 241ms/step - loss: 0.9487 - categorical_accuracy: 0.6031
Epoch 6/30
17/17 [==============================] - 4s 246ms/step - loss: 0.6312 - categorical_accuracy: 0.7784
Epoch 7/30
17/17 [==============================] - 4s 244ms/step - loss: 0.6193 - categorical_accuracy: 0.7611
Epoch 8/30
17/17 [==============================] - 4s 248ms/step - loss: 0.5189 - categorical_accuracy: 0.8112
Epoch 9/30
17/17 [==============================] - 4s 